In [1]:
import numpy as np
from itertools import combinations, product
#from scipy.sparse import csr_matrix

dim = 3 # Qudit dimension

In [2]:
def modadd(state, Z_or_X, index):
    if state[Z_or_X][index] == dim-1:
        state[Z_or_X][index]=0
    else:
        state[Z_or_X][index]+=1
    return state

def evolution(state, size):
    Z_part, X_part = state
    new_state = [[0 for _ in range(size)],[0 for _ in range(size)]]
    # X to Z evolution
    for i in range(size):
        for k in range(X_part[i]):
            # Z
            new_state = modadd(new_state, 0, i)
            new_state = modadd(new_state, 0, i)
            #new_state = modadd(new_state, 0, i) #4
    # Z evolution
    for i in range(size):
        for k in range(Z_part[i]):
            if i == 0: # Boundary
                # Z
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                #new_state = modadd(new_state, 0, i)#4
                new_state = modadd(new_state, 0, i+1)
                new_state = modadd(new_state, 0, i+1)
                #new_state = modadd(new_state, 0, i+1)#4
                # X
                new_state = modadd(new_state, 1, i)
            elif i < size-1:
                # Z
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i-1)
                #new_state = modadd(new_state, 0, i-1)#4
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                #new_state = modadd(new_state, 0, i)#4
                new_state = modadd(new_state, 0, i+1)
                new_state = modadd(new_state, 0, i+1)
                #new_state = modadd(new_state, 0, i+1)#4
                # X
                new_state = modadd(new_state, 1, i)
            else:
                # Z
                new_state = modadd(new_state, 0, i-1)
                new_state = modadd(new_state, 0, i-1)
                #new_state = modadd(new_state, 0, i-1)#4
                new_state = modadd(new_state, 0, i)
                new_state = modadd(new_state, 0, i)
                #new_state = modadd(new_state, 0, i)#4
                # X
                new_state = modadd(new_state, 1, i) 
    return new_state

In [11]:
state_0=[[1, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0]] # (Z, X)

size = len(state_0[0])

state = evolution(state_0, size)

all_states = [state_0]

while state != state_0:
    all_states.append(state)
    state = evolution(state, size)
print(len(all_states))


36


In [56]:
all_states=[[[1, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0]], [[1, 0, 1, 0, 0, 0, 0], \
[1, 1, 0, 0, 0, 0, 0]]]+\
[[[1, 0, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0]], [[1, 1, 0, 0, 0, 0, 0], \
[1, 0, 1, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0]]]+\
[[[1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0]]]

In [57]:
all_states

[[[1, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0]],
 [[1, 0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0]],
 [[1, 0, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0]],
 [[1, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0]],
 [[1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0]],
 [[1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0]]]

In [58]:
ortho = tuple([(tuple(i[0]), tuple(i[1])) for i in all_states]) # vectorised collection of all_states
# ortho = A_k as output

ortho_str = [''.join([str(elem) for elem in all_states[i][0]])\
 +''.join([str(elem) for elem in all_states[i][1]])for i in range(len(all_states))] # String rep.
# allows us to generate the indexes

new_ortho = [] # Allows us to detect wether commutation generate new Pauli matrices
# Will reduce combination size

size = len(all_states[0][0]) # Qudit count
new_length = len(ortho)

possible_decimal = [0 for i in range((dim**2)**size)] # List of all possible indexes of
# Pauli matrices. Value = 1 at some index meaning that specific Pauli matrix
# is contained in ortho

for i in ortho_str: # Set = 1 for pauli matrices in initial ortho
    ortho_index  = int(i, dim)
    if possible_decimal[ortho_index] == 0:
        possible_decimal[ortho_index] = 1

        
loop = True

while loop == True:
    comb1 = list(combinations([i for i in range(new_length)], 2))
    list1 = [i for i in range(new_length)]
    list2 = [i for i in range(new_length, len(ortho))]
    comb2 = list(product(list1,list2))
    total_comb = comb1 + comb2 # Desired combinations to generate commutations
    print("Length: ", len(total_comb), " list1: ", len(list1)\
         , " list2: ", len(list2))
    for i1, i2 in total_comb: # Iterate over desired comb.
        M1 = ortho[i1]
        M2 = ortho[i2]
        conj = [[0 for k in range(size)],[0 for k in range(size)]] #Z,X
        count = 0
        for i in range(size):
            # Below commutation based on notes
            a, b, c, d = M1[0][i], M1[1][i], M2[0][i], M2[1][i]
            Z_c = (a+c)%dim
            X_c = (b+d)%dim
            count += -b*c+d*a
            conj[0][i] = Z_c
            conj[1][i] = X_c
        if count%dim != 0: # equals 0 means [M1, M2] = 0
            conj = (tuple(conj[0]), tuple(conj[1]))
            conj_str = ''.join([str(elem) for elem in conj[0]])\
            +''.join([str(elem) for elem in conj[1]]) # Get hexadecimal rep. of vector
            conj_index = int(conj_str, dim) # Index of Pauli, convert from dim rep. to decimal
            if possible_decimal[conj_index] == 0: # Checks if ortho contains Pauli
                new_ortho.append(conj)
                possible_decimal[conj_index] = 1
                if (len(new_ortho)+len(ortho))%1000==0:
                    print(len(new_ortho)+len(ortho))
    if len(new_ortho) == 0: # If no new matrices generated from commutation, exit
        loop = False
        new_length = len(new_ortho)
    print('ortho: ', len(ortho), 'new: ', len(new_ortho))
    new_length = len(new_ortho)
    ortho = tuple(new_ortho) + ortho
    new_ortho = []
    #new_length =  # change to make faster, will be extremely large if len(new_ortho)
"""
while loop == True:
    for i1 in ortho:
        for i2 in ortho:
            M1 = i1
            M2 = i2
            conj = [[0 for k in range(size)],[0 for k in range(size)]] #Z,X
            count = 0
            for i in range(size):
                # Below commutation based on notes
                a, b, c, d = M1[0][i], M1[1][i], M2[0][i], M2[1][i]
                Z_c = (a+c)%dim
                X_c = (b+d)%dim
                count += -b*c+d*a
                conj[0][i] = Z_c
                conj[1][i] = X_c
            if count%dim != 0: # equals 0 means [M1, M2] = 0
                conj = (tuple(conj[0]), tuple(conj[1]))
                conj_str = ''.join([str(elem) for elem in conj[0]])\
                +''.join([str(elem) for elem in conj[1]]) # Get hexadecimal rep. of vector
                conj_index = int(conj_str, dim) # Index of Pauli, convert from dim rep. to decimal
                if possible_decimal[conj_index] == 0: # Checks if ortho contains Pauli
                    new_ortho.append(conj)
                    possible_decimal[conj_index] = 1
                    if (len(new_ortho)+len(ortho))%1000==0:
                        print(len(new_ortho)+len(ortho))
    if len(new_ortho) == 0: # If no new matrices generated from commutation, exit
        loop = False
        new_length = len(new_ortho)
    print('ortho: ', len(ortho), 'new: ', len(new_ortho))
    new_length = len(new_ortho)
    ortho = tuple(new_ortho) + ortho
    new_ortho = []
    #new_length =  # change to make faster, will be extremely large if len(new_ortho)
"""

print(len(ortho))

ortho:  7 new:  15
ortho:  22 new:  30
ortho:  52 new:  11
ortho:  63 new:  0
63
